# Исследовательский анализ данных заемщиков Банка

## Описание проекта:

#### Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

#### Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Описание данных:

In [13]:
import pandas as pd

In [14]:
data = pd.read_csv('D:\Yandex_Practikum\Credit_Dep\Project_Credit_Dep.csv')

data.info() # смотрим общую информацию по датасету

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Датасет содержит ~ 25 000 строк с данными о заемщиках Банка. 

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

#### Заказ от кредитного отдела проверить гипотезы:
- есть ли зависимость между количеством детей и возвратом кредита в срок?
- есть ли зависимость между семейным положением и возвратом кредита в срок?
- есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- как разные цели кредита влияют на его возврат в срок?

# Предобработка данных

### Работа с пропусками

In [15]:
data_na = data.isna().sum() # считаем сумму явных пропусков
print(data_na)

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


С помощью метода isna мы обнаружили наличие пропусков в столбцах:
- 'days_employed'(кол-во дней в труд.стаже, тип данных - float)
- 'total_income'(сумма кредита, тип данных - float).

Количество пропусков в обоих столбцах одинаково. Возможно эти пропуски взаимосвязаны. Проверим это с помощью фильтрации данных.

In [16]:
data_check_na = data.loc[data['days_employed'].isna()] # изучаем строки с пропусками

display(data_check_na.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


<div class="alert alert-success" role="alert">
Взаимосвязь пропусков очевидна. Возможные причины этих пропусков:
    
- это пул заявок не прошедших первоначальный скоринг, с неподтвержденными данными по занятости. Поэтому нет точных данных по стажу и сумме кредита, которые рассчитываются на более поздних этапах скоринга.
- часть из них возможно связаны с политикой Банка в отношении персональных данных некоторых категорий заемщиков.
</div>

In [17]:
print(data_na / len(data)) # считаем долю пропущенных значений от общей

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64


##### Предварительно исправим аномалию в данных столбца с трудовым стажем, большая часть значений в нем отрицательные. Переведем значения в положительные методом abs().  Возможно это ошибка при выгрузке данных, либо особенности учета в банковской системе.

In [18]:
data['days_employed'] = data['days_employed'].abs() 

Соотношение пропусков к общему количеству строк около 11%. 

Исходя из количественной категории данных, количества дней в трудовом стаже и ежемесячном доходе, пропуски для большей объективности данных лучше заполнить медианным значением. 

Т.к при подсчете среднего значения:

- максимальное значение (суммы больше миллиона) нескольких заемщиков может сильно сместить среднее значение к максимальному
- минимальные (в т.ч. пропуски, приравненные к 0) сильно сместить в сторону минимального
    
Подсчитаем медианное значение для столбцов с пропусками, и заполним этим значением пропуски.


#### Заполним медианным значением пропуски и проверим результаты

In [19]:
days_employed_median = pd.Series(data['days_employed']).median() #вычисляем медианное значение для стажа
total_income_median = pd.Series(data['total_income']).median() #вычисляем медианное значение для доходов

data['days_employed'] = data['days_employed'].fillna(value=days_employed_median) #заполняем медианным значением пропуски в стаже
data['total_income'] = data['total_income'].fillna(value=total_income_median) #заполняем медианным значением пропуски в доходах

print(data[data['days_employed'] == days_employed_median]['days_employed'].head(5))
print(data[data['total_income'] == total_income_median]['total_income'].head(5))

12    2194.220567
26    2194.220567
29    2194.220567
41    2194.220567
55    2194.220567
Name: days_employed, dtype: float64
12    145017.937533
26    145017.937533
29    145017.937533
41    145017.937533
55    145017.937533
Name: total_income, dtype: float64


Проверим остались ли необработанные пропуски

In [20]:
print(data.isna().sum()) 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Пропуски устранены.

#### Устраняем аномалии

Проверим есть ли еще аномалии в других столбцах с помощью метода unique() для перебора уникальных значений

In [21]:
print(data['children'].unique()) 

[ 1  0  3  2 -1  4 20  5]


<div class="alert alert-info">

Значения 20 и -1 явно аномальные
</div>

In [22]:
print(data['dob_years'].unique()) 

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]


<div class="alert alert-info">

Значение 0 также аномальное
</div>

In [23]:
print(data['education'].unique()) 

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


<div class="alert alert-info">

Присутствуют значения с разлиным регистром
</div>

In [24]:
print(data['family_status'].unique()) 

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


<div class="alert alert-info">

Присутствуют значения с разлиным регистром
</div>

In [25]:
print(data['income_type'].unique()) 

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


In [26]:
print(data['purpose'].unique()) 

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


<div class="alert alert-info">

Присутствуют неявные дубликаты, которые можно разбить на категории
</div>

In [27]:
print(data['days_employed'].max()) 
print(data['days_employed'].min()) 

data['days_employed'] = data['days_employed'].astype(int) 
data = data[data['days_employed'] <= 22000]

401755.40047533
24.14163324048118


<div class="alert alert-info">

- трудовой стаж вероятнее всего указан в календарных днях, поэтому максимальные значения аномальны.
- трудовой стаж в 65 лет составляет не более 22 000 дней, переведем значения в целочисленные, и отфильтруем аномальные значения.
</div>

In [28]:
print(data.groupby('gender')['gender'].count())

gender
F      11429
M       6650
XNA        1
Name: gender, dtype: int64


<div class="alert alert-info">

Присутствует одно аномальное значение, оставим его
</div>

#### В столбцах возраст и кол-во детей присутствуют аномальные значения. Вероятно они возникли при заполнении заемщиком анкеты и система скоринга обработала их, не приняв за ошибку.

##### Посчитаем их количество и решим, удалить их из выборки или предположить среднее значение, и им заполнить

In [29]:
print(data[data['children'] == 20]['children'].count()) 
print(data[data['children'] == -1]['children'].count())
print(data[data['dob_years'] == 0]['dob_years'].count())



69
40
84


<div class="alert alert-info">

- подсчет выдает 76, 47 и 101 строку с аномальным значением соответственно.
- строки со значением 20 и 0 отфильтруем, а значения -1 приведем в положительное значение с помощью abs()
</div>

In [30]:
data = data[data['children'] != 20]
data = data[data['dob_years'] != 0] 
data['children'] = data['children'].abs() 

### Замена вещественного типа данных на целочисленный

#### Сумма задолженности нам понадобится для дальнейшего анализа, поэтому изменим тип данных вещественного(дробного) на целочисленный int 

In [31]:
data['total_income'] = data['total_income'].astype(int) #приводим ежемесячный доход в целочисленное значение

<div class="alert alert-info">

В столбцах education и family_status присутствуют значения без единого (змеиного) регистра. Вероятно они возникли вследствие различного регистра при заполнении анкет заемщиками.
</div>

### Удаляем дубликаты

##### Для начала приведем их к единому регистру с помощью метода str.lower()

In [32]:
data['education'] = data['education'].str.lower() 
data['family_status'] = data['family_status'].str.lower() 

In [33]:
print(data.duplicated().sum()) 

71


In [34]:
data = data.drop_duplicates() 
print(data.duplicated().sum()) 

0


### Категоризация данных

In [35]:
education_id = data[['education','education_id']] #создадим два датафрейма education_id и family_status_id
family_status_id = data[['family_status', 'family_status_id']]

data.drop('education', axis=1, inplace=True) #удалим столбцы education и family_status из исходного датафрейма
data.drop('family_status', axis=1, inplace=True)

##### Разобъем заемщиков по категориям, исходя из уровня дохода

In [36]:
def income_category(row):
    if row['total_income'] <= 30000: #Категоризириуем данные столбца с уровнем дохода заемщика
        return 'E'
    if row['total_income'] <= 50000:
        return 'D'
    if row['total_income'] <= 200000:
        return 'C'
    if row['total_income'] <= 1000000:
        return 'B'
    return 'A' 

data['total_income_category'] = data.apply(income_category, axis=1)

#### Также по целям кредитования

In [37]:
def purpose_category(row):
    if 'авто' in row['purpose']: #Категоризириуем данные столбца с целями кредитования
        return 'операции с автомобилем'
    if 'недвиж' in row['purpose']:
        return 'операции с недвижимостью'
    if 'жил' in row['purpose']:
        return 'операции с недвижимостью'
    if 'свадьб' in row['purpose']:
        return 'проведение свадьбы'
    if 'образов' in row['purpose']:
        return 'получение образования'

data['purpose_category'] = data.apply(purpose_category, axis=1)

### Проверяем гипотезы

#### Зависимость возвратов кредита от количества детей

In [38]:
df_pivot_child = data.pivot_table(index='children', values='debt', aggfunc=['sum','count']) #создаем сводную таблицу по данным о количестве детей
df_pivot_child.reset_index()
df_pivot_child['children_ratio'] = df_pivot_child['sum'] / df_pivot_child['count'] * 100 # считаем долю проблемных займов по категориям

display(df_pivot_child)

,sum,count,children_ratio
,debt,debt,
children,,,
0,892,10885,8.194763
1,430,4579,9.390697
2,192,2022,9.495549
3,26,322,8.074534
4,4,40,10.000000
5,0,9,0.000000


In [39]:
data.groupby('children')['debt'].agg(['count','sum','mean'])

,count,sum,mean
children,,,
0,10885,892,0.081948
1,4579,430,0.093907
2,2022,192,0.094955
3,322,26,0.080745
4,40,4,0.100000
5,9,0,0.000000


In [40]:
def my_mean(x): return x.mean()*100

# В кач-ве агрегирования можно любую функцию брать.
data.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
0,10885,892,8.194763
1,4579,430,9.390697
2,2022,192,9.495549
3,322,26,8.074534
4,40,4,10.000000
5,9,0,0.000000


In [41]:
def my_mean(x): return '{:.2%} '.format(x.mean())

data.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
0,10885,892,8.19%
1,4579,430,9.39%
2,2022,192,9.50%
3,322,26,8.07%
4,40,4,10.00%
5,9,0,0.00%


<div class="alert alert-success" role="alert">

- Самая большая доля заемщиков не имеющих детей, от общей выборки более 60%
- Выборка заемщиков с 4 и 5 детьми крайне мала и ее процент возвратности мы не учитываем.
- Процент возвратности примерно одинаков, разница не более 2%. Но если учесть еще объем кредитования в денежном выражении. Разница даже в 1% может быть значительной.

Вывод: возвратность кредита от количества детей у заемщика варьируется в пределах 3%. При этом рост на 1% при наличии 1 и 2 детей возможно связан с:
    
- заемщики в этой категории чаще моложе и доходы в этом возрасте менее стабильны
- расходы значительно возрастают с появлением ребенка, при этом возможность использования материнского капитала для их снижения используется через 3 и более лет
- семьи с 3 и более детей чаще более зрелые и стабильные, в т.ч. относительно доходов
- заемщики, не имеющие детей, составляют наибольшую долю от общего числа заемщиков и явно больше других пользуются кредитными продуктами.

</div>

</p>.<span style="color:green">
    Зависимость возвратов кредита от семейного положения
    </span>

In [42]:
df_pivot_family_st = data.pivot_table(index='family_status_id', values='debt', aggfunc=['sum', 'count']) #создаем сводную таблицу по данным о семейном положении


df_pivot_family_st.columns = ['sum_debt', 'count_debt']
df_pivot_family_st = df_pivot_family_st.reset_index()
df_pivot_family_st['family_status_ratio'] = df_pivot_family_st['sum_debt'] / df_pivot_family_st['count_debt'] * 100 # считаем долю проблемных займов по категориям
family_status_id = family_status_id.drop_duplicates()
df_pivot_family_st = family_status_id.merge(df_pivot_family_st, how = 'outer', left_on='family_status_id', right_on='family_status_id') #объединяем датафрейм с таблицей со значениями family_status_id


display(df_pivot_family_st)


,family_status,family_status_id,sum_debt,count_debt,family_status_ratio
0,женат / замужем,0,826,10381,7.956844
1,гражданский брак,1,354,3542,9.994353
2,в разводе,3,73,986,7.403651
3,не женат / не замужем,4,260,2476,10.500808
4,вдовец / вдова,2,31,472,6.567797


<div class="alert alert-success" role="alert">

- Среди заемщиков больше всего женатых, от общей выборки более 60%, при этом в данной категории наименьший процент проблемных займов (если учесть количество заемщиков и не учитывать категорию "вдовцов")
- Наибольший процент невозвратности наблюдается у неженатых, и живущих в гражданском браке
- Разница в показателях заметна около 4%
- Банк выдает женатым в 2 раза больше кредитов, чем неженатым.
    
Вывод: зависимость доли проблемных займов от общей наблюдается. Женатые, в разводе и вдовцы имеют меньше проблем с кредитами, чем неженатые. Возможные причины:
    
- Женатые, в разводе и вдовцы более ответственные в распределении своих бюджетов и в обязательствах перед Банком
- Также у последних более стабильны: отношение к источникам своих доходов и семейные бюджеты

</div>

#### Зависимость возвратности кредита от уровня дохода заемщика (А - самый высокий, Е - самый низкий)

In [43]:
df_pivot_income = data.pivot_table(index='total_income_category', values='debt', aggfunc=['sum', 'count']) #создаем сводную таблицу по данным о доходах заемщиков

df_pivot_income.reset_index()
df_pivot_income['total_income_ratio'] = df_pivot_income['sum'] / df_pivot_income['count'] * 100 # считаем долю проблемных займов по категориям

display(df_pivot_income)

,sum,count,total_income_ratio
,debt,debt,
total_income_category,,,
A,2,25,8.000000
B,322,4447,7.240837
C,1206,13194,9.140518
D,14,184,7.608696
E,0,7,0.000000


<div class="alert alert-success" role="alert">

- Самая большая доля заемщиков категории "С"(со средним доходом), от общей выборки более 70%, при этом в данной категории наибольший процент проблемных займов.
- Выборка заемщиков категории "E"(с низким доходом) крайне мала и ее процент возвратности мы не учитываем.
- Процент возвратности примерно одинаков, разница менее 2% незначительна
    
    
Вывод: на первый вгляд возвратность кредита от уровня дохода заемщика незначительна. Однако данные по категории заемщиков со средним доходом говорят, что это наиболее проблемные заемщики. Возможные причины:
    
- их больше всего в кредитном портфеле Банка
- их кредитные лимиты довольно большие, что чаще негативно влияет на кредитную нагрузку заемщика
- данная категория уровня доходов наиболее нестабильная в своем диапазоне

</div>

#### Зависимость возвратности кредита от целей кредитования

In [44]:
df_pivot_purpose = data.pivot_table(index='purpose_category', values='debt', aggfunc=['sum', 'count']) #создаем сводную таблицу по данным о целях кредитования заемщиков

df_pivot_purpose.reset_index()
df_pivot_purpose['purpose_ratio'] = df_pivot_purpose['sum'] / df_pivot_purpose['count'] * 100 # считаем долю проблемных займов по категориям

display(df_pivot_purpose)

,sum,count,purpose_ratio
,debt,debt,
purpose_category,,,
операции с автомобилем,356,3559,10.002810
операции с недвижимостью,699,9039,7.733156
получение образования,327,3332,9.813926
проведение свадьбы,162,1927,8.406850


<div class="alert alert-success" role="alert">

- Самая большая доля заемщиков в категории "Операции с недвижимостью", от общей выборки около 50%
- При этом в категории "Операции с недвижимостью" наименьшая доля проблемных займов.
- Найбольшая доля проблемных кредитов в категориях "Операции с автомобилем" и "Получение образования"

Выводы:
    
- доля кредитного портфеля на категории "Операции с автомобилем" и "Получение образования" составляет чуть более 30% и имеет наибольший процент проблемных займов, хоть и незначительный.
    
Возможно это связано с влиянием на уровень расходов заемщика после получения кредита:
    
- после приобретения авто значительно возрастают расходы на топливо, обслуживание и страхование автомобиля, помимо платежей по кредиту
- получение образования обычно занимает много времени, т.е. от повышения\смены квалификации доход приходит нескоро, если вообще приходит. Плюс если заемщик берет кредит на собственное обучение, то это значительно повышает нагрузку на его работоспособность и ресурсоемкость, соответственно у последних чаще возникают нестабильность и форс-мажорные ситуации.
    
Меньший процент невозвратности по другим категориям возможен по причинам:
    
- "операции с недвижимостью" часто снижают расходы заемщика (исключая аренду жилья при приобретении своего, плюс у заемщика больше мотивации выкупать свое жилье, а не арендовать, займы на ремонт менее проблемны из-за сумм и размеров платежей)
- "проведение свадьбы" часто подразумевает досрочный возврат значительной суммы кредита за счет подарков в виде денежных средств

</div>

## Общий вывод:

#### Вышеизложенные категории имеют незначительные влияния на возвратность кредита заемщиком, если их учитывать по отдельности.

- заемщики без детей, за счет наибольшей доли в кредитном портфеле более 60%, можно назвать самыми проблемными, а с детьми более ответственными.

- Банк выдает больше всего кредитов женатым, более 60% от общего числа заемщиков, и как показало исследование последние более ответственны к обязательствам по платежам. А неженатые и проживающие в гражданском браке менее ответственны.

- по уровню дохода заемщики с высоким и низким доходом, несмотря на меньшую долю, более ответственны. Состоятельные чаще умеют управлять своими финансами, а люди с низким доходом часто имеют немного кредитов. А люди со средним доходом наиболее проблемные, т.к. чаще имеют много кредитов, соответственно бюджетом труднее управлять, плюс они составляют наибольший процент (более 70%) заемщиков как по числу, так и по объему кредитного портфеля.

- исходя из целей кредитования наиболее проблемными, с совокупной долей от портфеля около 30%, являются приобретение автомобиля и образовательных услуг. А наиболее ответственными заемщики, приобретающие жилье, и планирующие свадьбу. При чем ипотечные займы самые значительные по объему.


#### При совокупном применении всех категорий к заемщику и соответствующем анализе (с учетом сумм кредитных портфелей для каждой категории и их прибыльности), можно составить вполне показательный кредитный рейтинг для заемщиков.